# Entorno de trabajo: Crear un dataset de noticias de prensa a partir del motor de búsqueda Sophia

Este notebook muestra una manera de extraer noticias desde el motor de búsqueda de _Sophia_, a partir de una **palabra clave**, un **país** y un **periodo de tiempo**.


_Sophia_ es un proyecto I+i+D desarrollado en la Universidad Austral de Chile desde 2019. El motor de búsqueda se encuentra en fase de mejoramiento continuo. Por cualquier pregunta o problema técnica con este notebook, no dudar en contactarme por Discord (@MatthieuVernier) o por correo mvernier@inf.uach.cl.

## 0. Librerías útiles

- Elasticsearch es el motor de búsqueda utilizado por Sophia
- Requiere la instalación del paquete elasticsearch

In [1]:
!python -V

Python 3.9.7


In [2]:
!pip install elasticsearch==7.16.3

In [3]:
import elasticsearch
from datetime import datetime

In [4]:
elasticsearch.__version__

(7, 16, 3)

## 1. Parámetros de la búsqueda

Características del motor de búsqueda:  
- El formato de las fechas es: YYYY-MM-DD
- El motor de búsqueda incluye noticias por distintos paises, pero se recomienda utilizar "chile" por el momento.
- El motor de búsqueda incluye datos entre el año 2000 y el año 2021.

In [5]:
country="chile"
from_="2010-01-01"
to_="2021-12-31"
media_outlets=["elllanquihue","elaustral","laestrelladechiloe","elheraldoaustral","radiosago",
               "elrepuertero","elvacanudo","elhuemul","seminariolocal","elquellonino","elinsular",
               "radiopudeto","radioacogida","elcalbucano","segundos33",
               "prensadelestuario","fresiaahora","soychiloe"]

keyword="cambio climático"
simple_keyword=False

## 2. Parámetros de conexión al motor de búsqueda de Sophia

In [6]:
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS= "dummy_elastic"

In [7]:
#NO MODIFICAR

es = elasticsearch.Elasticsearch(
    IP,
    #port=PORT,
    http_auth=(USER, PASS)
)

## 3. Búsqueda de noticias de prensa

- Preparación de la consulta en el formato solicitado por Elasticsearch

In [8]:
#NO MODIFICAR
match=""
if (simple_keyword):
    match="match"
else:
    match="match_phrase"

query = { 
    "bool": { 
      "must": [
        {match: { "text":keyword}}

      ],
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }}},
        { "term":  { "country": country }},
        { "terms":  { "media_outlet": media_outlets }} 
    ]
    }  
}

- Envio de la consulta
- No se puede devolver más de 10.000 noticias. En caso de obtener 10.000 respuestas, se recomienda buscar en un periodo de tiempo más corto o modificar la consulta.

In [9]:
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 4393 noticias encontradas...


## 4. Guardar los datos en un dataframe Pandas y en un archivo CSV

In [10]:
import pandas as pd

data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[],'search':[]}

df = pd.DataFrame(data)  
  
for hit in res['hits']['hits']:
    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    search = keyword
    
    new_row = {'id_news':id_news, 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date, 'search':search}
    
    df = df.append(new_row, ignore_index=True)

df

,id_news,country,media_outlet,url,title,text,date,search
0,22581387.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2020/02/02/...,Encuesta revela qué pensamos sobre el cambio c...,El 93% de los latinoamericanos está de acuerdo...,2020-02-02,cambio climático
1,47473342.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2020/02/02/...,Encuesta revela qué pensamos sobre el cambio c...,El 93% de los latinoamericanos está de acuerdo...,2020-02-02,cambio climático
2,21425188.0,chile,elhuemul,https://www.elhuemul.cl/2015/11/30/futaleufu-s...,Futaleufú se adhiere a la marcha mundial contr...,Entre las actividades se desarrolló una marcha...,2015-11-30,cambio climático
3,22387263.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2019/1...,Cambio climático: qué piensan y sienten en tod...,El 93% de los latinoamericanos está de acuerdo...,2019-12-30,cambio climático
4,22580118.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2019/1...,Cambio climático: qué piensan y sienten en tod...,El 93% de los latinoamericanos está de acuerdo...,2019-12-30,cambio climático
...,...,...,...,...,...,...,...,...
4388,21451085.0,chile,elrepuertero,https://www.elrepuertero.cl/noticia/politica/l...,Los ejes que marcaron la Cuenta Pública 2020: ...,"Este viernes, a partir de las 20:00 horas, el ...",2020-08-01,cambio climático
4389,21456843.0,chile,elvacanudo,https://www.elvacanudo.cl/noticia/politica/los...,Los ejes que marcaron la Cuenta Pública 2020: ...,"Este viernes, a partir de las 20:00 horas, el ...",2020-08-01,cambio climático
4390,21410426.0,chile,radiopudeto,http://www.pudeto.cl/?p=43808,GREENPEACE REVELA EVIDENCIA CIENTÍFICA QUE IND...,Greenpeace dio a conocer un informe respecto d...,2016-09-05,cambio climático
4391,47395965.0,chile,elaustral,https://www.australosorno.cl/impresa/2017/01/0...,"""Gobernadores regionales deben ser elegidos es...",Los duros embates de 2016 parecen no haber ami...,2017-01-02,cambio climático


- Verifiquemos si existen noticias con la misma URL

In [11]:
df['url'].duplicated().any()

True

- Eliminar las noticias duplicadas

In [12]:
df = df.drop_duplicates(subset='url', keep='first')

In [13]:
df

,id_news,country,media_outlet,url,title,text,date,search
0,22581387.0,chile,elllanquihue,https://www.ellanquihue.cl/impresa/2020/02/02/...,Encuesta revela qué pensamos sobre el cambio c...,El 93% de los latinoamericanos está de acuerdo...,2020-02-02,cambio climático
2,21425188.0,chile,elhuemul,https://www.elhuemul.cl/2015/11/30/futaleufu-s...,Futaleufú se adhiere a la marcha mundial contr...,Entre las actividades se desarrolló una marcha...,2015-11-30,cambio climático
3,22387263.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2019/1...,Cambio climático: qué piensan y sienten en tod...,El 93% de los latinoamericanos está de acuerdo...,2019-12-30,cambio climático
6,21426965.0,chile,elhuemul,https://www.elhuemul.cl/2015/11/17/el-29-de-no...,El 29 de noviembre Futaleufu se sumará a la Ma...,"La iniciativa, propulsada en Futaleufu por el ...",2015-11-17,cambio climático
7,21398696.0,chile,elhuemul,https://www.elhuemul.cl/2019/11/18/comunas-de-...,Comunas de Palena y Chaitén preparan seminario...,El cambio climático es el mal de nuestro tiemp...,2019-11-18,cambio climático
...,...,...,...,...,...,...,...,...
4388,21451085.0,chile,elrepuertero,https://www.elrepuertero.cl/noticia/politica/l...,Los ejes que marcaron la Cuenta Pública 2020: ...,"Este viernes, a partir de las 20:00 horas, el ...",2020-08-01,cambio climático
4389,21456843.0,chile,elvacanudo,https://www.elvacanudo.cl/noticia/politica/los...,Los ejes que marcaron la Cuenta Pública 2020: ...,"Este viernes, a partir de las 20:00 horas, el ...",2020-08-01,cambio climático
4390,21410426.0,chile,radiopudeto,http://www.pudeto.cl/?p=43808,GREENPEACE REVELA EVIDENCIA CIENTÍFICA QUE IND...,Greenpeace dio a conocer un informe respecto d...,2016-09-05,cambio climático
4391,47395965.0,chile,elaustral,https://www.australosorno.cl/impresa/2017/01/0...,"""Gobernadores regionales deben ser elegidos es...",Los duros embates de 2016 parecen no haber ami...,2017-01-02,cambio climático


- Guardar los archivos en un CSV

In [14]:
nombre_archivo=keyword.replace(' ', '')+"_"+country+"_"+from_+"_"+to_+".csv"
print("El archivo se llama:")
print(nombre_archivo)
df.to_csv("./"+nombre_archivo)

El archivo se llama:
cambioclimático_chile_2010-01-01_2021-12-31.csv


- Realizar algunas consultas SQL sobre el dataset

In [16]:
!pip install pandasql
from pandasql import sqldf 

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26781 sha256=9aa5a8fd6f46906b0278ea115133089f4f51d3100d7b4fb9091d1edd4b12e910
  Stored in directory: c:\users\marce\appdata\local\pip\cache\wheels\63\e8\ec\75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql


In [17]:
result = sqldf("SELECT media_outlet,count(*) FROM df GROUP BY media_outlet ORDER BY count(*) DESC")
result

,media_outlet,count(*)
0,elaustral,660
1,elllanquihue,554
2,laestrelladechiloe,421
3,elvacanudo,238
4,elrepuertero,230
5,elheraldoaustral,223
6,radiosago,142
7,elhuemul,90
8,radiopudeto,42
9,seminariolocal,19
